# 02 Regime Long-Short Demo
Risk-on vs risk-off macro regimes and a biotech/pharma long-short example.

- **Risk-on**: markets price in growth and liquidity; lower rates, rising equities, moderate volatility.
- **Risk-off**: stress or uncertainty; higher volatility and weaker equities.
- **Long-short**: long one asset and short another; here long biotech (XBI) vs short pharma (XPH) in risk-on regimes.
- **Intuition**: biotech tends to outperform when risk appetite is healthy, while pharma is more defensive.

In [ ]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ensure repo root on path for src imports
repo_root = Path.cwd().resolve()
if not (repo_root / 'src').exists():
    repo_root = repo_root.parent
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

from src.data.etf_loader import load_clean_prices
from src.signals.regime import compute_monthly_features, classify_regime
from src.signals.ls_biotech_pharma import build_monthly_ls_weights
from src.backtest.engine import run_backtest
from src.analysis.metrics import compute_cagr, compute_annual_vol, compute_sharpe, compute_max_drawdown

%matplotlib inline


In [ ]:
prices = load_clean_prices()[['XBI', 'XPH', 'SPY']]
prices = prices.dropna(how='any')
prices.head()


### Placeholder macro series
For this demo we synthesize simple macro series. Replace with real TNX/VIX/SPY data loaders later.

In [ ]:
idx = prices.index
# Synthetic 10Y yield oscillating around 3%
tnx_yield = pd.Series(0.03 + 0.005 * np.sin(np.linspace(0, 6, len(idx))), index=idx, name='TNX')
# VIX proxy tied loosely to SPY volatility (higher when SPY moves more)
spy_ret = prices['SPY'].pct_change().fillna(0)
vix_level = 20 + 80 * spy_ret.abs().rolling(21, min_periods=1).mean()
vix = vix_level.clip(lower=10, upper=60)
# Use SPY adjusted prices directly
spy_prices = prices['SPY']


In [ ]:
monthly_features = compute_monthly_features(tnx_yield, spy_prices, vix)
regime_labels = classify_regime(monthly_features)
regime_labels.tail()


In [ ]:
ls_weights_daily = build_monthly_ls_weights(regime_labels, prices.index)
ls_weights_daily.head()


In [ ]:
# Backtest on the XBI/XPH slice
price_slice = prices[['XBI', 'XPH']]
bt_result = run_backtest(price_slice, ls_weights_daily)
bt_result.equity_curve.tail()


In [ ]:
cagr = compute_cagr(bt_result.daily_returns)
ann_vol = compute_annual_vol(bt_result.daily_returns)
sharpe = compute_sharpe(bt_result.daily_returns)
max_dd = compute_max_drawdown(bt_result.equity_curve)
print(f'CAGR:   {cagr:.2%}')
print(f'Vol:    {ann_vol:.2%}')
print(f'Sharpe: {sharpe:.2f}')
print(f'Max DD: {max_dd:.2%}')


In [ ]:
# Long-only comparators
xbi_equity = (1 + price_slice['XBI'].pct_change().fillna(0)).cumprod()
xph_equity = (1 + price_slice['XPH'].pct_change().fillna(0)).cumprod()

fig, ax = plt.subplots(figsize=(12, 6))
bt_result.equity_curve.plot(ax=ax, label='LS: +XBI / -XPH')
xbi_equity.plot(ax=ax, label='Long XBI')
xph_equity.plot(ax=ax, label='Long XPH')
ax.set_title('Equity Curves: Biotech vs Pharma Long-Short')
ax.set_ylabel('Cumulative Wealth')
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Regime shading
regime_daily = regime_labels.reindex(price_slice.index, method='ffill').fillna(0)
fig, ax = plt.subplots(figsize=(12, 3))
price_slice['XBI'].plot(ax=ax, color='steelblue', label='XBI price (adj)')
ax.set_ylabel('Price')
ax.set_title('Risk-On (shaded) vs XBI Price')
# Highlight risk-on spans using blocks where regime == 1
in_on = regime_daily == 1
on_changes = in_on.ne(in_on.shift()).cumsum()
for _, series in regime_daily.groupby(on_changes):
    if series.iloc[0] == 1:
        ax.axvspan(series.index[0], series.index[-1], color='green', alpha=0.1)
ax.legend()
plt.tight_layout()
plt.show()


## Robustness: threshold sweep
Quick grid over rate/VIX/SPY thresholds.

In [ ]:
from src.analysis.robustness import sweep_regime_parameters

rate_grid = [-1.0, -0.5, 0.0]
vix_grid = [20, 25, 30]
spy_grid = [0.0]
sweep_df = sweep_regime_parameters(prices[['XBI','XPH']], tnx_yield, spy_prices, vix, rate_grid, vix_grid, spy_grid)
sweep_df.sort_values('sharpe', ascending=False).head()


Interpretation: look for stability—similar Sharpe across nearby thresholds implies robustness; large swings suggest sensitivity.